<a href="https://colab.research.google.com/github/WaiWasabi/Neural-Networks/blob/master/NeuralNetworkFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import random

def sigmoid(z):
  return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z)*(1 - sigmoid(z))

def cost_derivative(aL, y): # where aL is the vector of output activations and y is the vector of expected outputs.
  return aL-y

class Network(object):
  def __init__(self, sizes):
    self.num_layers = len(sizes)
    self.sizes = sizes
    self.biases = [np.random.randn(x,1) for x in sizes[1:]]
    self.weights = [np.random.randn(x,y) for x, y in zip(sizes[1:], sizes[:-1])]

  def feedforward(self, a): # mostly used for testing samples.
    for w, b in zip(self.weights, self.biases):
      a = sigmoid(np.dot(w, a) + b)
    return a
  
  def forward_pass(self, x, y):
    activation = x
    activations = [x]
    zs = []

    for b, w in zip(self.biases, self.weights):
      z = np.dot(w, activation) + b
      zs.append(z)
      activation = sigmoid(z)
      activations.append(activation)

    return activations, zs, y

  def backprop(self, forward_pass):
    activations, zs, y = forward_pass
    """import data from a forward pass"""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    """initiate gradients"""
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    for l in range(2, self.num_layers):
      z = zs[-l]
      delta = np.dot(self.weights[-l+1].transpose(), delta) * sigmoid_prime(z)
      nabla_b[-l] = delta
      nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

    return(nabla_b, nabla_w)

  def update_mini_batch(self, mini_batch, lr): 
    """with mini_batch being a list of tuples (x, y),
    where x is a training example and y is expected output,
    and lr being the learning rate."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]

    for x, y in mini_batch:
      delta_nabla_b, delta_nabla_w = self.backprop(self.forward_pass(x,y))
      nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
      nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.biases = [b-(lr/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
    self.weights = [w-(lr/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
  
  def SGD(self, train_data, epochs, mini_batch_size, lr, test_data = None):
    n = len(train_data)
    for i in range(epochs):
      random.shuffle(train_data)
      mini_batches = [train_data[k : k + mini_batch_size] for k in range(0, n, mini_batch_size)]
      for j in mini_batches:
        self.update_mini_batch(j, lr)
      #print(f"Epoch {i} complete")